In [ ]:
#Este programa deve ler o arquivo .csv com dados de mercado e portar 
#estes dados para o elasticsearch

"""
*Modelo de estrutura do dado:
{
"_index":"marketdata-btcusdt-binance",
"_type":"marketdata", --> Type será depreciado do projeto elasticsearch*
        *https://www.elastic.co/guide/en/elasticsearch/reference/master/removal-of-types.html
"_id":"<timestamp>",
"_version":1,
"_seq_no":0,
"_primary_term":1,
"found":true,
    "_source":{
    "Open":"3839.94", 
    "High":"3839.98", 
    "Low":"3735.0", 
    "Close":"3735.0", 
    "Volume":"24.85793", 
    "N_of_trades":"129.0"
    }
}

"""

In [137]:
#!/usr/bin/env python3

#Importando bibliotecas

#import elasticsearch_dsl
from elasticsearch_dsl import Document, Float
from elasticsearch_dsl.connections import connections

import pandas as pd

import sys
import os

from datetime import datetime
import time

In [87]:
#Lendo os dados .csv

csv_data_path = os.getcwd().split(sep="notebook_files/local_tests/db_test")[0] +'local_tests/db_test/'

market_data = pd.read_csv(csv_data_path+'dados.csv', index_col='Open_time', parse_dates=['Open_time','Close_time'], infer_datetime_format=True)    

In [141]:
connections.create_connection(hosts=['localhost'])

class Candle(Document):
    Open = Float()
    High = Float()
    Low = Float()
    Close = Float()
    Volume = Float()
    N_of_trades = Float()
    
    class Index:
        name = 'marketdata-btcusdt-binance'
    
    def save(self, **kwargs):
        return super(Candle, self).save(**kwargs)
    
Candle.init()

for i in range(0,len(market_data)):
    candle = Candle(meta={'id': time.mktime((market_data.index[i]).timetuple())})
    candle.Open = float(market_data.Open[i])
    candle.High = float(market_data.High[i])
    candle.Low = float(market_data.Low[i])
    candle.Close = float(market_data.Close[i])
    candle.Volume = float(market_data.Volume[i])
    candle.N_of_trades = float(market_data.N_of_trades[i])
   
    candle.save()